In [ ]:
## 위클리 검수 현황 엑셀로 다운로드
# 실행(▶ Run)하고 위클리아이디를 입력하시면 해당하는 위클리 검색 현황을 엑셀로 다운받을수있습니다.
# 다운받은 엑셀파일은 검수현황폴더 내에 저장됩니다.
# 엑셀파일의 파일명은 "위클리아이디_suggestion_다운받은날짜" 입니다.
# 엑셀파일을 클릭한 후 Download를 클릭하시면 다운로드됩니다.

In [ ]:
import pandas as pd
import datetime
import sys
sys.path.append('/home/libs')
config = __import__('config')

weekly_id = input("위클리 아이디")

In [ ]:
cur = config.conn.cursor()
cur.execute("""
    select d.id,
       CASE
           WHEN d.confirmation_status = 1 THEN '매칭 성공'
           WHEN d.confirmation_status = 2 THEN '제안서 전송'
           WHEN d.confirmation_status = 3 THEN '피드백 진행중'
           WHEN d.confirmation_status = 4 THEN '완료'
           WHEN d.confirmation_status = 5 THEN '피드백 없이 종료'
           ELSE ''
           END                                    as confirmation_status,
       d.notice,
       TO_CHAR(d.matching_at, 'YYYY-MM-DD hh:mm') as matching_at,
       TO_CHAR(d.send_at, 'YYYY-MM-DD hh:mm')     as send_at,
       TO_CHAR(d.done_at, 'YYYY-MM-DD')     as done_at,
       CASE WHEN d.is_confirmed = true THEN '검수 완료' ELSE '' END as is_confirmed,
       concat('https://stylerecipe-korea.s3.amazonaws.com/media',d.thumbnail) as thumbnail
from styling_weeklyinfo a
         join styling_weeklystylinginfo sw on a.id = sw.weekly_info_id
         join "Request_request" c on sw.request_id = c.id
         join "Suggestion_suggestion" d on c.suggestion_id = d.id
where a.id = %s
group by d.id
""", (weekly_id,))

queryResult = cur.fetchall()

df = pd.DataFrame(queryResult, columns = 
    [
        "제안서 번호",
        "상태",
        "요청사항",
        "매칭 일자",
        "제안서 전송 일자",        
        "완료 일자",
        "검수 여부",
        "썸네일"
    ]
)

current_date = datetime.datetime.now().strftime('%Y%m%d')

df.to_excel('./검수현황/'+weekly_id+'_suggestion_'+current_date+'.xlsx', index=False)

print("엑셀 다운로드 완료")
config.conn.close()